In [ ]:
import torch
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
import torch.nn as nn

In [ ]:
cls_embeddings = torch.load("models/cls_embeddings.pt")         # shape: [1267, 768]
knowledge_tensor = torch.load("models/knowledge_tensor.pt")     # shape: [1267, 128]

df = pd.read_csv("Data/liar_dataset/test.tsv", sep='\t', header=None)
raw_labels = df[1].tolist()

label_map = {
    "true": 1, "mostly-true": 1, "half-true": 1,
    "barely-true": 0, "false": 0, "pants-fire": 0
}
labels = [label_map[label.strip().lower()] for label in raw_labels]
labels = torch.tensor(labels, dtype=torch.long)

In [ ]:
class FusionClassifier(nn.Module):
    def __init__(self, bert_dim=768, knowledge_dim=128, hidden_dim=256, num_classes=2):
        super(FusionClassifier, self).__init__()
        self.fusion = nn.Sequential(
            nn.Linear(bert_dim + knowledge_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, cls_embedding, knowledge_vector):
        fused = torch.cat((cls_embedding, knowledge_vector), dim=1)
        return self.fusion(fused)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FusionClassifier().to(device)
model.load_state_dict(torch.load("models/fusion_model.pt"))
model.eval()

In [ ]:
dataset = TensorDataset(cls_embeddings, knowledge_tensor, labels)
loader = DataLoader(dataset, batch_size=32)

In [ ]:
all_preds = []
all_labels = []

with torch.no_grad():
    for cls_batch, k_batch, y_batch in loader:
        cls_batch = cls_batch.to(device)
        k_batch = k_batch.to(device)

        outputs = model(cls_batch, k_batch)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(y_batch.tolist())

In [ ]:
output_df = pd.DataFrame({
    "label": all_labels,
    "prediction": all_preds
})
output_df.to_csv("outputs/fusion_predictions.csv", index=False)
print("✅ Predictions saved to: outputs/fusion_predictions.csv")

In [ ]:
acc = accuracy_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
print(f"📊 Final Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")

✅ Predictions saved to: outputs/fusion_predictions.csv
📊 Final Accuracy: 0.5675 | F1 Score: 0.7181


C:\Users\mca\AppData\Local\Temp\ipykernel_3936\3441243999.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cls_embeddings = torch.load("models/cls_embeddings.pt")       